LDA's approach to topic modeling is to consider each document as a collection of topics and each topic as a collection of keywords. By providing the algorithm with a number of topics, it is able to rearrange the topic distribution within documents and the keyword distribution within the topics to create an optimal composition of the topic-keyword distribution.

Essentially, topics are collections of prominent keywords or words with the highest probability in the topic, which helps to identify the subject matter of the topics.

## Installing dependencies


In [2]:
!pip3 install spacy

!python3 -m spacy download en #Language model

!pip3 install gensim # For topic modeling

!pip3 install pyLDAvis # For visualizing topic models

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-23 00:33:04.111074: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 55.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/s

For this implementation we will be using stopwords from NLTK.

In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

## Importing the necessary libraries

In [5]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
import pyLDAvis.gensim_models as gensimvis

## Preparation of stopwords
You can extend the list of stopwords depending on the dataset you are using or if you see any stopwords even after preprocessing. You can also visualize your cleaned corpus using wordcloud and check if any words are adding noise or there are any stopwords still left out in the cleaned corpus.

In [7]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

## Loading Dataset
We will be 20-Newsgroups dataset. It contains about 11K news group post from 20 different topics. Dataset is available at newsgroup.json:https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json

In [9]:
# LoadDataset
df=pd.read_json('/content/drive/MyDrive/filename/newsgroups.json')
print(df.target_names.unique())
df.head()

['rec.autos' 'comp.sys.mac.hardware' 'comp.graphics' 'sci.space'
 'talk.politics.guns' 'sci.med' 'comp.sys.ibm.pc.hardware'
 'comp.os.ms-windows.misc' 'rec.motorcycles' 'talk.religion.misc'
 'misc.forsale' 'alt.atheism' 'sci.electronics' 'comp.windows.x'
 'rec.sport.hockey' 'rec.sport.baseball' 'soc.religion.christian'
 'talk.politics.mideast' 'talk.politics.misc' 'sci.crypt']


,content,target,target_names
0,From: lerxst@wam.umd.edu (where's my thing)\nS...,7,rec.autos
1,From: guykuo@carson.u.washington.edu (Guy Kuo)...,4,comp.sys.mac.hardware
2,From: twillis@ec.ecn.purdue.edu (Thomas E Will...,4,comp.sys.mac.hardware
3,From: jgreen@amber (Joe Green)\nSubject: Re: W...,1,comp.graphics
4,From: jcm@head-cfa.harvard.edu (Jonathan McDow...,14,sci.space


## Remove emails and newline characters
* As you can see there are lot of emails and newline characters present in the dataset. we can remove them using regular expression. By using the function sub() of re module. IN re.sub() specify a regular expression pattern in the first argument, a new string in the second argument, and a string to be processed in the third argument.

In [10]:
# Convert to list 
data = df.content.values.tolist()  
# Remove Emails 
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]  
# Remove new line characters 
data = [re.sub('\s+', ' ', sent) for sent in data]  
# Remove distracting single quotes 
data = [re.sub("\'", "", sent) for sent in data]  
pprint(data[:1])

<>:4: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<>:4: DeprecationWarning: invalid escape sequence \S
<>:6: DeprecationWarning: invalid escape sequence \s
<ipython-input-10-b5167ec19e8c>:4: DeprecationWarning: invalid escape sequence \S
  data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]
<ipython-input-10-b5167ec19e8c>:6: DeprecationWarning: invalid escape sequence \s
  data = [re.sub('\s+', ' ', sent) for sent in data]


['From: (wheres my thing) Subject: WHAT car is this!? Nntp-Posting-Host: '
 'rac3.wam.umd.edu Organization: University of Maryland, College Park Lines: '
 '15 I was wondering if anyone out there could enlighten me on this car I saw '
 'the other day. It was a 2-door sports car, looked to be from the late 60s/ '
 'early 70s. It was called a Bricklin. The doors were really small. In '
 'addition, the front bumper was separate from the rest of the body. This is '
 'all I know. If anyone can tellme a model name, engine specs, years of '
 'production, where this car is made, history, or whatever info you have on '
 'this funky looking car, please e-mail. Thanks, - IL ---- brought to you by '
 'your neighborhood Lerxst ---- ']


The text still looks messy , so we carry on further  with our preprocessing.

## Tokenize words and cleanup the text
We use  gensims simple_preprocess(), set deacc=True to remove punctuations.

In [11]:
def sent_to_words(sentences):
  for sentence in sentences:
    yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))            #deacc=True removes punctuations
data_words = list(sent_to_words(data))
print(data_words[:1])

[['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp', 'posting', 'host', 'rac', 'wam', 'umd', 'edu', 'organization', 'university', 'of', 'maryland', 'college', 'park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front', 'bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']]


## Creating Bigram and Trigram models
Bigrams are 2 words frequently occuring together in docuent. Trigrams are 3 words frequently occuring.The 2 arguments for Phrases are min_count and threshold. The higher the values of these parameters , the harder its for a word to be combined to bigram.

In [12]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.8/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['from', 'wheres', 'my', 'thing', 'subject', 'what', 'car', 'is', 'this', 'nntp_posting_host', 'rac_wam_umd_edu', 'organization', 'university', 'of', 'maryland_college_park', 'lines', 'was', 'wondering', 'if', 'anyone', 'out', 'there', 'could', 'enlighten', 'me', 'on', 'this', 'car', 'saw', 'the', 'other', 'day', 'it', 'was', 'door', 'sports', 'car', 'looked', 'to', 'be', 'from', 'the', 'late', 'early', 'it', 'was', 'called', 'bricklin', 'the', 'doors', 'were', 'really', 'small', 'in', 'addition', 'the', 'front_bumper', 'was', 'separate', 'from', 'the', 'rest', 'of', 'the', 'body', 'this', 'is', 'all', 'know', 'if', 'anyone', 'can', 'tellme', 'model', 'name', 'engine', 'specs', 'years', 'of', 'production', 'where', 'this', 'car', 'is', 'made', 'history', 'or', 'whatever', 'info', 'you', 'have', 'on', 'this', 'funky', 'looking', 'car', 'please', 'mail', 'thanks', 'il', 'brought', 'to', 'you', 'by', 'your', 'neighborhood', 'lerxst']


## Remove Stopwords, make bigrams and lemmatize
* Using lemmatization instead of stemming is a practice which especially pays off in topic modeling because lemmatized words tend to be more human-readable than stemming.

In [13]:
# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

## Call the functions in order

In [22]:
!python3 -m spacy download en

/usr/local/lib/python3.8/dist-packages/torch/cuda/__init__.py:497: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
2023-01-23 01:13:16.202568: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 49.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [23]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en_core_web_sm' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en_core_web_sm')

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:1])

[['s', 'thing', 'car', 'nntp_poste', 'host', 'park', 'line', 'wonder', 'enlighten', 'car', 'see', 'day', 'door', 'sport', 'car', 'look', 'late', 'early', 'call', 'door', 'really', 'small', 'addition', 'separate', 'rest', 'body', 'know', 'tellme', 'model', 'name', 'engine', 'spec', 'year', 'production', 'car', 'make', 'history', 'info', 'funky', 'look', 'car', 'mail', 'thank', 'bring', 'neighborhood', 'lerxst']]


## Create Dictionary and Corpus needed for Topic Modeling
* Making  sure that we check if dictionary[id2word] or corpus is clean otherwise we may not get good quality topics.

In [24]:
# Create Dictionary 
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 5), (5, 1), (6, 2), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 2), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1)]]


* Gensim creates unique id for each word in the document. Its mapping of word_id and word_frequency. Example: (18,2) above indicates, word_id 18 occurs twice in the document and so on.
* This is used as input to LDA model.

Readable format of corpus can be obtained by executing the code below:

In [25]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('addition', 1),
  ('body', 1),
  ('bring', 1),
  ('call', 1),
  ('car', 5),
  ('day', 1),
  ('door', 2),
  ('early', 1),
  ('engine', 1),
  ('enlighten', 1),
  ('funky', 1),
  ('history', 1),
  ('host', 1),
  ('info', 1),
  ('know', 1),
  ('late', 1),
  ('lerxst', 1),
  ('line', 1),
  ('look', 2),
  ('mail', 1),
  ('make', 1),
  ('model', 1),
  ('name', 1),
  ('neighborhood', 1),
  ('nntp_poste', 1),
  ('park', 1),
  ('production', 1),
  ('really', 1),
  ('rest', 1),
  ('s', 1),
  ('see', 1),
  ('separate', 1),
  ('small', 1),
  ('spec', 1),
  ('sport', 1),
  ('tellme', 1),
  ('thank', 1),
  ('thing', 1),
  ('wonder', 1),
  ('year', 1)]]

## Building topic model
> Parameters of LDA

* Alpha and Beta are Hyperparameters — alpha represents document-topic density and Beta represents topic-word density, chunksize is the number of documents to be used in each training chunk, update_every determines how often the model parameters should be updated and passes is the total number of training passes.
* A measure for best number of topics really depends on kind of corpus you are using, the size of corpus, number of topics you expect to see.

In [26]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt

View topics in LDA model

* Each topic is combination of keywords and each keyword contributes a certain weightage to the topic.
* we can see keywords for each topic and weightage of each keyword using lda_model.print_topics().

In [27]:
# Print the keyword of topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.024*"christian" + 0.023*"god" + 0.021*"physical" + 0.021*"book" + '
  '0.020*"man" + 0.010*"direct" + 0.009*"bible" + 0.009*"love" + '
  '0.009*"choose" + 0.008*"evil"'),
 (1,
  '0.026*"car" + 0.015*"buy" + 0.012*"price" + 0.011*"cheap" + 0.010*"bike" + '
  '0.009*"cost" + 0.009*"rate" + 0.009*"high" + 0.009*"slow" + 0.009*"drive"'),
 (2,
  '0.016*"internet" + 0.015*"color" + 0.013*"use" + 0.013*"reference" + '
  '0.012*"tape" + 0.012*"type" + 0.012*"size" + 0.010*"section" + '
  '0.010*"monitor" + 0.010*"page"'),
 (3,
  '0.037*"_" + 0.018*"c" + 0.017*"m" + 0.015*"rlk" + 0.010*"sc" + 0.008*"ei" + '
  '0.008*"mc" + 0.008*"cx" + 0.008*"cj" + 0.007*"frequency"'),
 (4,
  '0.019*"evidence" + 0.017*"reason" + 0.017*"believe" + 0.013*"claim" + '
  '0.011*"sense" + 0.010*"exist" + 0.009*"faith" + 0.009*"mean" + 0.008*"true" '
  '+ 0.008*"talk"'),
 (5,
  '0.048*"team" + 0.046*"game" + 0.029*"play" + 0.028*"win" + 0.021*"entry" + '
  '0.012*"cool" + 0.012*"character" + 0.012*"score" + 

* wou can see the top keywords and weights associated with keywords contributing to topic.
* Topics are words with highest probability in topic and the numbers are the probabilities of words appearing in topic distribution.
* But looking at keywords can you guess what the topic is?
* we may summarize topic-4 as evidence(In the above figure). Each one may have different topic at particular number , topic 4 might not be in the same place where it is now, it may be in topic 10 or any number.

## Evaluating our  topic models
## We compute model Perplexity and Coherence score
Coherence score and perplexity provide a convinent way to measure how good a given topic model is.

In [28]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  
# a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.8/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt


Perplexity:  -8.308226480899608

Coherence Score:  0.5226704870124077


according to the output we can deduce that our perplexity is lower by a value of -8 which shows the model is performing better since lower perplexity implies data is more likely and the topic coherence is 0.5 which is basic-good in terms of being  human interpretable.

## Visualizing the topic model


In [31]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

# feed the LDA model into the pyLDAvis instance
vis = gensimvis.prepare(lda_model, corpus, id2word)
vis

/usr/local/lib/python3.8/dist-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
9     -0.299573 -0.086625       1        1  19.015913
11    -0.224082 -0.063221       2        1  11.033980
4     -0.173531 -0.081463       3        1   8.747282
12    -0.140022  0.053166       4        1   7.081390
14    -0.113875  0.116076       5        1   6.734493
16    -0.107057  0.244405       6        1   5.496415
13    -0.052624 -0.243904       7        1   5.103144
6      0.160289 -0.034938       8        1   4.914722
1     -0.055769  0.099924       9        1   4.908863
15    -0.009430 -0.156308      10        1   4.251141
2      0.034023  0.055022      11        1   3.609806
19    -0.041541  0.172313      12        1   3.561108
5      0.095750 -0.038150      13        1   2.697648
0      0.066148 -0.123576      14        1   2.634299
7      0.106077  0.102302      15        1   2.273266
10     0.134275 -0.002375      16        1   2.006153
18     0.124595  0.068502      17        1   1.924419
17     0.123764 -0.040329      18        1   1.910914
8      0.178242 -0.025309      19        1   1.180932
3      0.194339 -0.015511      20        1   0.914111, topic_info=               Term          Freq         Total Category  logprob  loglift
4322             ax  49508.000000  49508.000000  Default  30.0000  30.0000
17             line  12481.000000  12481.000000  Default  29.0000  29.0000
204           write   9789.000000   9789.000000  Default  28.0000  28.0000
142    organization   6708.000000   6708.000000  Default  27.0000  27.0000
24       nntp_poste   5098.000000   5098.000000  Default  26.0000  26.0000
...             ...           ...           ...      ...      ...      ...
47932           umu     55.149683     69.207668  Topic20  -5.3447   4.4679
1072             sc    116.761657    231.456650  Topic20  -4.5947   4.0107
343               m    191.186992   2351.186604  Topic20  -4.1015   2.1855
188               i     43.081450    682.049149  Topic20  -5.5917   1.9330
3756             ad     30.283238     92.785038  Topic20  -5.9442   3.5753

[1013 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
4319      20  0.996987            _
13327      8  0.986974     abnormal
5452      12  0.993281  accelerator
1342       1  0.690285       accept
1342      14  0.309613       accept
...      ...       ...          ...
613        2  0.357358        young
613        7  0.597323        young
613       18  0.044886        young
5296      19  0.988119        youth
2590      15  0.994444         zone

[1641 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[10, 12, 5, 13, 15, 17, 14, 7, 2, 16, 3, 20, 6, 1, 8, 11, 19, 18, 9, 4])

Each bubble on the left-hand side represents topic. The larger the bubble, the more prevalent or dominant the topic is. Good topic model will be fairly big topics scattered in different quadrants rather than being clustered on one quadrant.

* The model with too many topics will have many overlaps, small sized bubbles clustered in one region of chart.
*If we/you move the cursor of  the different bubbles we/you can see different keywords associated with topics.

## How can we find optimum number of topics?
* One approach to find optimum number of topics is build many LDA models with different values of number of topics and pick the one that gives highest coherence value.
* If we see the same keywords being repeated in multiple topics, it’s probably a sign that the ‘k’ is too large.
* Sometimes topic keyword may not be enough to make sense of what topic is about. So for better understanding of topics, we can find the documents a given topic has contributed the most to and infer the topic by reading the documents.
